In [1]:
#Code borrowed from:
# https://www.learndatasci.com/glossary/tf-idf-term-frequency-inverse-document-frequency/#:~:text=As%20its%20name%20implies%2C%20TF,of%20words%20in%20the%20document.
# ChatGPT
# https://medium.com/codex/document-indexing-using-tf-idf-189afd04a9fc



import nltk
from nltk.corpus import state_union
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
import numpy as np

nltk.download('state_union')

documents = [state_union.raw(fileid) for fileid in state_union.fileids()]

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(documents)

terms = vectorizer.get_feature_names_out()

inverted_index = defaultdict(list)

for i, term in enumerate(terms):
    for j in range(len(documents)):
        tfidf_score = tfidf_matrix[j, i]
        if tfidf_score > 0:
            inverted_index[term].append((j, tfidf_score))

inverted_index = {term: tuple(postings) for term, postings in inverted_index.items()}


# for term, postings in inverted_index.items():
#     print(f"{term}: {postings}")


[nltk_data] Downloading package state_union to
[nltk_data]     C:\Users\Aidan\AppData\Roaming\nltk_data...
[nltk_data]   Package state_union is already up-to-date!


In [8]:
# Code Borrowed From:
# https://github.com/Tixierae/deep_learning_NLP/blob/master/other/inverted_index_tfidf.ipynb
# ChatGPT



from collections import Counter

def convert_query_to_vector(query_terms, inverted_index, total_documents):
    query_vector = {}

    df_query_terms = Counter(doc_idx for term in query_terms for doc_idx, _ in inverted_index.get(term, []))

    for term in query_terms:
        df = df_query_terms[term] if term in df_query_terms else 0
        idf = np.log((total_documents + 1) / (df + 1)) + 1  
        query_vector[term] = idf

    return query_vector

query_terms = ["president", "liberation", "office"]
total_documents = len(documents)

query_vector = convert_query_to_vector(query_terms, inverted_index, total_documents)
# print("Query Vector:", query_vector)


Query Vector: {'president': 5.189654742026425, 'liberation': 5.189654742026425, 'office': 5.189654742026425}


In [10]:
import numpy as np

def cosine_similarity(query_vector, inverted_index, total_documents):
    doc_scores = {}

    for term, idf_query in query_vector.items():
        postings = inverted_index.get(term, [])

        for doc_idx, tfidf_doc in postings:
            if doc_idx not in doc_scores:
                doc_scores[doc_idx] = 0

            doc_scores[doc_idx] += idf_query * tfidf_doc

    for doc_idx, score in doc_scores.items():
        doc_vector_length = np.linalg.norm([tfidf for _, tfidf in inverted_index['document']])
        doc_scores[doc_idx] /= doc_vector_length

    sorted_results = sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)

    return sorted_results

query_terms = ["president", "liberation", "office"]
total_documents = len(documents)

query_vector = convert_query_to_vector(query_terms, inverted_index, total_documents)

search_results = cosine_similarity(query_vector, inverted_index, total_documents)

for doc_idx, score in search_results:
    print(f"Document {doc_idx}: Score {score}")


Document 0: Score 15.155671057261221
Document 33: Score 14.545688256757964
Document 30: Score 8.84912463750947
Document 25: Score 8.43714926661777
Document 21: Score 8.297223458026583
Document 31: Score 7.406505434364755
Document 51: Score 6.271774470727011
Document 48: Score 6.164262374738004
Document 20: Score 5.892003522434546
Document 23: Score 5.859224886699166
Document 54: Score 5.75332638941217
Document 15: Score 5.647650115406831
Document 43: Score 5.641697822557677
Document 44: Score 5.415612934272736
Document 64: Score 5.413906366168448
Document 59: Score 5.191927286714122
Document 45: Score 5.139676493170534
Document 28: Score 5.111327760272521
Document 24: Score 5.050243641751988
Document 63: Score 5.029189461696802
Document 22: Score 5.007128788613372
Document 32: Score 4.988985387889361
Document 46: Score 4.8731657816048415
Document 17: Score 4.7395699388683745
Document 50: Score 4.658174559219636
Document 49: Score 4.632613602963814
Document 52: Score 4.521802266564763
D